Se establecen las condiciones iniciales y se hacen los cálculos fijos. Después se definen varias funciones. La función principal es main, que realiza la evolución del sistema.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from numpy import random as randomp
from random import seed
from random import random
from datetime import datetime
from numba import jit

#semilla para num aleatorios
now = datetime.now()
seed(now.microsecond)
#randomp.seed(now.microsecond)



#paso 0 (inicializar) globales

npart=20
paso=npart*npart
p=20
sp=randomp.choice([0,1],size=(npart,npart)) 


#patron
patron1=np.loadtxt("patron1.txt")
uno=np.loadtxt("unouno.txt")
dos=np.loadtxt("dosdos.txt")
tres=np.loadtxt("tres.txt")
cuatro=np.loadtxt("cuatro.txt")
#patron1=randomp.choice([0,1],size=(npart,npart))
zeta=randomp.choice([0,1],size=(p,npart,npart))
###patrones prefijados
# zeta[0]=uno
# zeta[1]=dos
# zeta[2]=tres
# zeta[3]=cuatro
# for k in range(npart):
#     for j in range(npart//2):
#         zeta[0][k][j+15]=randomp.choice([0,1],size=1)
#         zeta[1][k][j]=randomp.choice([0,1],size=1)
# for k in range(npart//2):
#     for j in range(npart):        
#         zeta[2][k+15][j]=randomp.choice([0,1],size=1)
#         zeta[3][k][j]=randomp.choice([0,1],size=1)



#patron a archivo
g=open("patron.dat","w")
for j in range(npart):
    for k in range(npart):
        if(k!=npart-1):
            print(str(zeta[0][j][k])+", ",end="", file=g)
        else:
            print(zeta[0][j][k],end="", file=g)
    print("\n",end='',file=g)
    
g.close()

h=open("salida0.dat","w")
for j in range(npart):
    for k in range(npart):
        if(k!=npart-1):
            print(str(sp[j][k])+", ",end="", file=h)
        else:
            print(sp[j][k],end="", file=h)
    print("\n",end='',file=h)
        
h.close()



theta=np.zeros((npart,npart))
                
                
                
    
##################suma de a
a=np.zeros(p)
for l in range(p):
    for i in range (npart):
        for j in range (npart):
            a[l]=a[l]+zeta[l][i][j]      
a=a/(npart*npart)
        
@jit(nopython=True)  ##El @jit es un de paquete que permite no tener que cargar las funciones en cada llamada,
def omega(i,j,k,l):  ##si no que se asemejan a como funcionan en C++, optimizando considerablemente el tiempo de cálculo.
    suma=0
    if ((i!=k) or (j!=l)):
        for n2 in range (p):
            suma=suma+(zeta[n2][i][j]-a[n2])*(zeta[n2][k][l]-a[n2])
    return suma/(npart*npart)
omeg=np.zeros((npart,npart,npart,npart))
for i in range(npart):
    for j in range(npart):
        for k in range(npart):
            for l in range(npart):
                omeg[i][j][k][l]=omega(i,j,k,l)
#umbral de disparo:(calculamos una vez todos los umbrales de disparo para no tener que calcularlos en cada iteración,
# ya que es fijo a partir de los patrones) 
for i in range(npart):
    for j in range(npart):
        for k in range(npart):
            for l in range(npart):
                theta[i][j]=theta[i][j]+omeg[i,j,k,l]
theta=0.5*theta                
###################


######### calculo del hamiltoniano #######
@jit(nopython=True)
def hamilton1(s):
    suma1=0
    suma2=0
    for i in range (npart):
        for j in range (npart):
            suma2=suma2+theta[i][j]*s[i][j]
            for k in range(npart):
                for l1 in range(npart):
                    suma1=suma1+omeg[i,j,k,l1]*s[i][j]*s[k][l1]

    return -0.5*suma1+suma2
#########################################
@jit(nopython=True)
def hamilton(s,a,b):
    suma=0
    for i in range(npart):
        if(i!=a):
            for j in range(npart):
                if(j!=b):
                    suma=suma+omeg[i][j][a][b]*s[i][j]
                
    return (1-2*s[a][b])*(theta[a][b]-0.5*omeg[a][b][a][b]-suma)


########### calculo del solapamiento
@jit(nopython=False)
def solapamiento(s,pos):
    suma=0
    for i in range(npart):
        for j in range(npart):
            suma=suma+(zeta[pos][i][j]-a[pos])*(s[i][j]-a[pos])
    
    return suma/(npart*npart*a[pos]*(1-a[pos]))


############################
def main(t,montec,difuminado,dif):
#variables de la función
    itera=montec*paso #numero iteraciones

    s=np.copy(sp)

    if(difuminado==True):  #Para modificar el estado inicial y hacer que fuese el patrón difuminado
        for i in range(npart):
            for j in range(npart):
                chi=random()
                if(chi<dif):
                    s[i][j]=zeta[0][i][j]

    
    positera=np.zeros((itera//paso+1))
    gsolap=np.zeros((p,itera//paso+1))

    for i in range(itera+1):
        n = randomp.randint(npart)
        m = randomp.randint(npart)


      
        minimo=min(1,np.exp(-hamilton(s,n,m)/t))

      

        xi=random()
       

        if(xi<minimo):
            if(s[n][m]==0):
                s[n][m]=1
            else:
                s[n][m]=0
        
        ###Aquí se guardan la red en cada paso monte carlo para luego poder estudiar la evolución
        if(i%paso==0):
            positera[i//paso]=i/paso
            for k in range(p):
                gsolap[k][i//paso]=(solapamiento(s,k)) 
        
        
        if(i%paso==0):
            f=open("salida"+str(i//paso+1)+".dat","w")
            #print(s) 
            for j in range(npart):
                for k in range(npart):
                    if(k!=npart-1):
                        print(str(s[j][k])+", ",end="", file=f)
                    else:
                        print(s[j][k],end="", file=f)

                print("\n",end='',file=f)

                #print("\n",end='',file=f)

    
    f.close()
    
    vecsolap=np.zeros(p)
    for i in range(p):
        vecsolap[i]=solapamiento(s,i)

    
    #return vecsolap ----------- Para estudiar el solapamiento final
    #return ((positera,gsolap) ----------- Para estudiar el solapamiento en cada paso Monte Carlo
    #el return de la función se ha modificado dependiendo de la gráfica que se quería realizar

#%prun main()